In [1]:
%load_ext autoreload
%autoreload 2

# Large Language Models Playing Mixed Strategy Nash Equilibrium Games

Alonso Silva (Nokia Bell Labs) 

LINCS Annual Workshop - June 27th, 2024.

## Table of contents

- Introduction to Large Language Models (LLMs)
- Introduction to Mixed Strategy Nash Equilibrium Games
- LLMs playing Mixed Strategy Nash Equilibrium Games
- Conclusions

# Introduction to Large Language Models

## Completion

In [40]:
prompt = "One, two, three,"

In [41]:
import requests

options = {"prompt": prompt, "temperature": 0, "seed": 0}
api_completions = "http://localhost:8000/v1/completions"
response = requests.post(api_completions, json=options).json()
print(response["choices"][0]["text"])

 four, five, six, seven, eight, nine, ten.




## Chat

![ChatGPT UI](./assets/ChatGPT.jpg "ChatGPT")

In [50]:
visible_user_prompt = "What's the capital of France?"

In [53]:
messages = [
    {"role": "user", "content": visible_user_prompt},
]
options_chat = {"messages": messages, "temperature": 0, "seed": 0, "max_tokens": None}
api_chat_completions = "http://localhost:8000/v1/chat/completions"
response = requests.post(api_chat_completions, json=options_chat).json()
response["choices"][0]["message"]

{'content': ' The capital of France is Paris.', 'role': 'assistant'}

In [44]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
prompt = tokenizer.apply_chat_template(messages, tokenize=False)
prompt

"<s>[INST] What's the capital of France?[/INST]"

In [55]:
prompt = f"<s>[INST] What's the capital of France? [/INST]"
options = {'prompt': prompt, 'temperature': 0, 'seed': 0, 'max_tokens': None}
response = requests.post(api_completions, json=options).json()
print(response['choices'][0]['text'])

 The capital of France is Paris.


## Function calling

![ChatGPT](./assets/ChatGPT-Use-Python.jpg)

In [56]:
visible_user_prompt = "What's 12345 times 54321?"

In [57]:
multiply_function = {
    "type": "function",
    "function": {
        "name": "multiply",
        "description": "Multiply two integers together.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"description": "First integer", "type": "integer"},
                "b": {"description": "Second integer", "type": "integer"},
            },
            "required": ["a", "b"],
        },
    },
}

In [58]:
import json

final_prompt = (
    "<s>[AVAILABLE_TOOLS] ["
    + json.dumps(multiply_function)
    + "] [/AVAILABLE_TOOLS][INST] "
    + visible_user_prompt
    + " [/INST]"
    + "[TOOL_CALLS] "
)
final_prompt

'<s>[AVAILABLE_TOOLS] [{"type": "function", "function": {"name": "multiply", "description": "Multiply two integers together.", "parameters": {"type": "object", "properties": {"a": {"description": "First integer", "type": "integer"}, "b": {"description": "Second integer", "type": "integer"}}, "required": ["a", "b"]}}}] [/AVAILABLE_TOOLS][INST] What\'s 12345 times 54321? [/INST][TOOL_CALLS] '

In [59]:
options = {'prompt': final_prompt, 'temperature': 0, 'seed': 0, 'max_tokens': None}
response = requests.post(api_completions, json=options).json()
print(response['choices'][0]['text'])

 [{"name": "multiply", "arguments": {"a": 12345, "b": 54321}}]


In [60]:
import regex

def extract_first_of_nested_brackets(input_string):
    # Pattern to match the first complete set of brackets including nested ones
    pattern = r'\{(?:[^\{\}]*|(?R))*\}'
    match = regex.search(pattern, input_string)
    if match:
        return match.group(0)  # Returns the matched text including brackets
    return None

In [61]:
extract_first_of_nested_brackets(response['choices'][0]['text'])

'{"name": "multiply", "arguments": {"a": 12345, "b": 54321}}'

In [62]:
from langchain_core.pydantic_v1 import BaseModel, Field

class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [63]:
from langchain_core.utils.function_calling import convert_to_openai_tool

json.dumps(convert_to_openai_tool(multiply))

'{"type": "function", "function": {"name": "multiply", "description": "Multiply two integers together.", "parameters": {"type": "object", "properties": {"a": {"description": "First integer", "type": "integer"}, "b": {"description": "Second integer", "type": "integer"}}, "required": ["a", "b"]}}}'

In [22]:
import json

from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_experimental.tools.python.tool import PythonAstREPLTool

tool = PythonAstREPLTool()
function = convert_to_openai_tool(tool)
json.dumps(function)

'{"type": "function", "function": {"name": "python_repl_ast", "description": "A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.", "parameters": {"type": "object", "properties": {"query": {"description": "code snippet to run", "type": "string"}}, "required": ["query"]}}}'

In [23]:
user_prompt = "What's 12345 times 54321?"

In [24]:
import json

final_prompt = (
    "<s>[AVAILABLE_TOOLS] ["
    + json.dumps(function)
    + "] [/AVAILABLE_TOOLS][INST] "
    + user_prompt
    + " [/INST]"
    + "[TOOL_CALLS] "
)
final_prompt

'<s>[AVAILABLE_TOOLS] [{"type": "function", "function": {"name": "python_repl_ast", "description": "A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.", "parameters": {"type": "object", "properties": {"query": {"description": "code snippet to run", "type": "string"}}, "required": ["query"]}}}] [/AVAILABLE_TOOLS][INST] What\'s 12345 times 54321? [/INST][TOOL_CALLS] '

In [25]:
options = {'prompt': final_prompt, 'temperature': 0, 'seed': 42, 'max_tokens': None}
response = requests.post(api_completions, json=options).json()
print(response['choices'][0]['text'])

 [{"name": "python_repl_ast", "arguments": {"query": "print(12345 * 54321)"}}]


In [26]:
clean_response = extract_first_of_nested_brackets(response['choices'][0]['text'])
clean_response

'{"name": "python_repl_ast", "arguments": {"query": "print(12345 * 54321)"}}'

In [27]:
import json_repair
output = json_repair.loads(clean_response)['arguments']['query']
print(tool.run(output))

670592745



## Introduction to Mixed Strategy Nash Equilibrium Games

## Matching Pennies: Game description

- Each player has two possible actions: to play Heads or Tails.
- Both players reveal their choices simultaneously.
- If both choices match (both Heads or both Tails), Player 1 wins. If the choices do not match (one Heads and one Tails), Player 2 wins.

## Matching Pennies: Payoff Matrix

|       | Heads   | Tails   |
|-------|---------|---------|
| Heads | (1,-1)  | (-1,1)  |
| Tails | (-1,1)  | (1,-1)  |

## Matching Pennies: Motivation Example

- Alice and Bob need to communicate sensitive information securely over a network that offers two distinct communication channels (let's call them Heads and Tails).
- However, their communication is at risk as Eve, an eavesdropper, aims to intercept their messages. Eve has the capability to spy on only one of these channels.

## Matching Pennies: Nash equilibrium

- No pure strategy Nash equilibrium
- Mixed strategy Nash equilibrium: each player chooses Heads or Tails with equal probability (50\% chance of Heads and 50\% chance of Tails).
- No incentive to deviate from the 50-50 strategy, which makes it a Nash equilibrium.

## Modified Matching Pennies: Payoff Matrix

|       | Heads   | Tails   |
|-------|---------|---------|
| Heads | (7,-1)  | (-1,1)  |
| Tails | (-1,1)  | (1,-1)  |

## Modified Matching Pennies: Nash Equilibrium
- Player 1 plays both options (Heads and Tails) with equal probability $1/2$.
- Player 2 plays Heads with probability $1/5$ and Tails with probability $4/5$.

## LLMs playing Mixed Strategy Nash Equilibrium Games

### Does the LLM know the game?

![ChatGPT](./assets/ChatGPT-Nothing.jpg)

[Link to full conversation](https://chatgpt.com/share/8a611112-43c3-4054-8faf-c5bde813fb3f)

PROMPT: "What's the Mixed Strategy Nash equilibrium for the matching pennies game?"

ANSWER: The Matching Pennies game is a two-player, zero-sum game where each player secretly chooses either heads (H) or tails (T). If both players choose the same option, the player who chose first wins the coin, and if they choose differently, the second player wins.

In a Nash equilibrium, neither player has an incentive to deviate from their strategy if they believe the other player will stick to theirs. However, in the Matching Pennies game, there is no pure strategy Nash equilibrium because both players would have an incentive to deviate from their strategies if they knew what the other player was going to do.

Instead, the Matching Pennies game has a mixed strategy Nash equilibrium. In this case, each player should randomly choose heads or tails with equal probability (0.5). This way, neither player can gain an advantage by deviating from their random choice, as they have no information about the other player's decision.

**So, the mixed strategy Nash equilibrium for the Matching Pennies game is each player choosing H and T with a 50% probability.**

### Does the LLM play the game well?

![ChatGPT](./assets/ChatGPT-MP.jpg)

In [28]:
user_prompt = (
    "Respond to the following text with one of the following answers: `heads`, `tails`. "
    "Provide your response in JSON format containing a single key `answer`. "
    "**Text**: ```Let's play matching pennies. I have made my choice. "
    "Now, you should make your choice. What do you play?\n```"
)

In [29]:
from json_repair import repair_json

answers = []
for seed in range(5):
    options = {
        'messages': [{"role": "user", "content": user_prompt}],
        'seed': seed,
    }
    response_raw = requests.post(f"http://localhost:8000/v1/chat/completions", json=options).json()
    response = response_raw["choices"][0]["message"]["content"]
    repaired_response = repair_json(response)
    first_of_nested_brackets = extract_first_of_nested_brackets(repaired_response)
    answer = json_repair.loads(first_of_nested_brackets)["answer"]
    answers.append(answer)
    print(answer)

heads
heads
heads
tails
heads


| Probability          | Value  |
|----------------------|--------|
| $p_{\text{Heads}}$   | 77.9%  |
| $p_{\text{Tails}}$   | 22.1%  |

## Let LLMs run code

![ChatGPT](./assets/ChatGPT-MP-Use-Python.jpg)

In [30]:
user_prompt = (
    "Let's play matching pennies. I have made my choice. Now, you should make your choice. "
    "What do you play? Use Python."
)

In [31]:
final_prompt = (
    "<s>[AVAILABLE_TOOLS] ["
    + str(function)
    + "] [/AVAILABLE_TOOLS][INST] "
    + user_prompt
    + " [/INST]"
    + """[TOOL_CALLS] [{"name": "python_repl_ast", "arguments": {"query": """
)

In [32]:
options = {"prompt": final_prompt, "max_tokens": None, "temperature": 0, "seed": 42,}

In [33]:
response = requests.post(f"http://localhost:8000/v1/completions", json=options).json()
text = """{'name': 'python_repl_ast', 'arguments': {'query':""" + f"""{str(response['choices'][0]['text'][:-1])}"""
print(json_repair.loads(text)['arguments']['query'])

import random
choice = random.randint(0, 1)
print('Heads' if choice == 0 else 'Tails')


In [34]:
import random
choice = random.randint(0, 1)
print('Heads' if choice == 0 else 'Tails')

Heads


## Can the LLM play a small modification of the game?

![ChatGPT](./assets/ChatGPT-Final.jpg)

[Link to full conversation](https://chatgpt.com/share/fc5f2a12-c50f-4ece-bbe2-6a898b645353)

In [35]:
user_prompt = (
    "Let's play once a modification of the Matching Pennies game. "
    "More precisely, each of us has two possible actions: to play Heads or Tails. "
    "We both reveal our choices simultaneously. "
    "If both of us play Heads, then I gain 7 points and you lose 1 point, "
    "if both of us play Tails, then I gain 1 point and you lose 1 point. "
    "If our choices do not match (one Heads and one Tails), you gain 1 point and I lose 1 point. "
    "I have made my choice. Now, you should make your choice. "
    "What do you play? Choose your choice first in valid Python code."
)

In [36]:
final_prompt = "<s>[AVAILABLE_TOOLS] [" + str(function) + "] [/AVAILABLE_TOOLS][INST] " + \
                user_prompt + " [/INST]" + "[TOOL_CALLS] [{\'name\': \'python_repl_ast\', \'arguments\': {\'query\': "

In [37]:
options = {"prompt": final_prompt, "max_tokens": None, "temperature": 0, "seed": 42}

In [38]:
response = requests.post(f"http://localhost:8000/v1/completions", json=options).json()
text = """{'name': 'python_repl_ast', 'arguments': {'query':""" + f"""{str(response['choices'][0]['text'][:-1])}"""
first_of_nested_brackets = extract_first_of_nested_brackets(text)
print(json_repair.loads(first_of_nested_brackets)['arguments']['query'])

import random
choice = 'Heads' if random.randint(0, 1) == 0 else 'Tails'
print('I play: ', choice)


In [39]:
import random
choice = "Heads" if random.randint(0, 1) == 0 else "Tails"
print(choice)

Heads


## Conclusions and Perspectives

- Despite their preexisting knowledge of classic games, LLMs are bad at playing Mixed Strategy Nash Equilibrium games. Perhaps [not surprising](https://x.com/alonsosilva/status/1679948389813821443).

- LLMs can significantly enhance their performance when they are enabled to run code and are encouraged to use this functionality. See also [GPT-4o](https://chatgpt.com/share/bf481a05-c6a7-4d8f-8474-9bd77329aa16)

- Critical limitations in the adaptability of LLMs when confronted with games that involve slightly more complex randomization strategies or slight modifications from standard scenarios.

- In such cases, the performance of LLMs noticeably declines, suggesting that while LLMs are proficient in handling familiar and straightforward game dynamics, their effectiveness is reduced in more complex or altered game setups.

- The paper can be found in [arXiv](https://arxiv.org/abs/2406.10574) and all code of the paper can be found [here](https://github.com/alonsosilvaallende/LLMs_Playing_MSNE_Games/tree/main)

Thank you!